In [ ]:
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
from openbb import obb

In [ ]:
obb.user.preferences.output_type = "dataframe"

In [ ]:
dax_tickers = [
    "ADS.DE", "ALV.DE", "BAS.DE", "BAYN.DE", "BEI.DE", "BMW.DE", "BNR.DE", "CON.DE",
    "1COV.DE", "DAI.DE", "DHER.DE", "DB1.DE", "DBK.DE", "DTE.DE", "EOAN.DE", "FME.DE",
    "FRE.DE", "HEI.DE", "HEN3.DE", "IFX.DE", "LIN.DE", "MRK.DE", "MTX.DE", "MUV2.DE",
    "PUM.DE", "QIA.DE", "RWE.DE", "SAP.DE", "SIE.DE", "SY1.DE", "SHL.DE", "VNA.DE",
    "VOW3.DE", "ZAL.DE", "MBG.DE", "HNR1.DE", "SRT3.DE", "NDA.DE", "DWNI.DE"
]

In [ ]:
def get_stock_data(ticker):
    return obb.equity.price.historical(ticker, start_date="2014-01-01", provider="yfinance")

get_stock_data("ADS.DE")

In [ ]:
fundamentals_df = obb.equity.fundamental.income(symbol="ADS.DE", provider="yfinance")

In [ ]:
fundamentals_df

In [ ]:
data = obb.equity.fundamental.income(
    symbol="AAPL", 
    provider="yfinance", 
    period="quarter", 
    limit=5  # 10 years of data (quarterly)
)

df = data

# Look for columns like this:
if "weighted_average_basic_shares_outstanding" in df.columns:
    df = df[["date", "weighted_average_basic_shares_outstanding"]]
    df["date"] = pd.to_datetime(df["date"])
    df = df.set_index("date").sort_index()
    df.plot(title="Shares Outstanding Over Time")

In [ ]:
df

In [ ]:
def get_shares_outstanding(ticker):
    try:
        fundamentals_df = obb.equity.fundamental.balance(ticker=ticker, period="annual", limit=150)
        if fundamentals_df is None or fundamentals_df.empty:
            return None
        shares_df = fundamentals_df[fundamentals_df["Metric"].str.contains("Shares", case=False)]
        shares_df = shares_df.set_index("Metric").T
        shares_df.index = pd.to_datetime(shares_df.index)
        shares_df.columns = ["Shares Outstanding"]
        return shares_df
    except Exception as e:
        print(f"Failed to get shares for {ticker}: {e}")
        return None
    
get_shares_outstanding("ADS.DE")

In [ ]:
# Step 3: Get shares outstanding data


# Step 4: Detect buybacks
def detect_buybacks(shares_df):
    shares_df = shares_df.dropna().sort_index()
    shares_df['change'] = shares_df['Shares Outstanding'].pct_change()
    buybacks = shares_df[shares_df['change'] < -0.01]  # >1% drop
    return buybacks

# Step 5: Calculate forward returns
def calculate_returns(ticker, buyback_dates):
    returns = []
    price_df = get_stock_data(ticker)
    price_df = price_df.set_index('date')
    price_df.index = pd.to_datetime(price_df.index)

    for date in buyback_dates:
        date = pd.to_datetime(date)
        base_price = price_df.loc[date:date + timedelta(days=15)]['adj_close'].head(1).values
        if base_price.size == 0:
            continue
        base_price = base_price[0]

        def get_future_return(months):
            future_date = date + pd.DateOffset(months=months)
            future_price = price_df.loc[future_date:future_date + timedelta(days=15)]['adj_close'].head(1).values
            if future_price.size == 0:
                return None
            return (future_price[0] - base_price) / base_price

        returns.append({
            "ticker": ticker,
            "buyback_date": date,
            "return_3m": get_future_return(3),
            "return_6m": get_future_return(6),
            "return_12m": get_future_return(12)
        })

    return returns

# Step 6: Get SPY returns
def get_spy_returns(reference_dates):
    spy = yf.download("SPY", start="2014-01-01", interval="1mo")['Adj Close']
    spy.index = pd.to_datetime(spy.index)
    returns = []
    for date in reference_dates:
        date = pd.to_datetime(date)
        if date not in spy.index:
            date = spy.index[spy.index.get_loc(date, method='bfill')]
        base_price = spy.loc[date]

        def get_return(months):
            future_date = date + pd.DateOffset(months=months)
            future_date = spy.index[spy.index.get_loc(future_date, method='bfill')]
            return (spy.loc[future_date] - base_price) / base_price

        returns.append({
            "date": date,
            "spy_return_3m": get_return(3),
            "spy_return_6m": get_return(6),
            "spy_return_12m": get_return(12),
        })
    return pd.DataFrame(returns)

# Main pipeline
def run_analysis():
    results = []
    dax_tickers = get_dax_tickers()

    for ticker in dax_tickers:
        try:
            shares_df = get_shares_outstanding(ticker)
            if shares_df is None or shares_df.empty:
                continue
            buybacks = detect_buybacks(shares_df)
            if buybacks.empty:
                continue
            returns = calculate_returns(ticker, buybacks.index)
            results.extend(returns)
        except Exception as e:
            print(f"Error processing {ticker}: {e}")

    result_df = pd.DataFrame(results)
    if not result_df.empty:
        spy_comp = get_spy_returns(result_df['buyback_date'])
        merged = pd.merge(result_df, spy_comp, left_on='buyback_date', right_on='date', how='left')
        return merged

# Run and export to CSV
df_result = run_analysis()
if df_result is not None:
    df_result.to_csv("dax_buybacks_vs_spy.csv", index=False)
    print(df_result.head())
else:
    print("No buybacks found.")